# Introduction
I've noticed in the model that there is no glutamine and arginine. Also in the version that was originally sent to me from UBath these metabolites were not present, though Beata did determine their presence in biomass. As in another notebook I saw that glutamate formation is somehow a problem when certain changes are made, I want to check if this is already the case in our model before those changes. 

1) I will add the glutamine and arginine into the biomass reaction and see what their effect may be on the growth rate. Then I can determine maybe better where the problem lies. 

2) I will remove this DNA/RNA node (seperately) and see which metabolites become problematic in creating biomass. I've looked into this before after the mentioned changes, but then there are 22 metabolites that are a problem. Hopefully at this stage there are some less and then it helps pinpoint the issues more.


In [2]:
import cameo
import pandas as pd
import cobra.io
import escher
from escher import Builder
from cobra import Reaction

In [3]:
model = cobra.io.read_sbml_model('../model/p-thermo.xml')

In [3]:
model_e_coli = cameo.load_model('iML1515')

In [4]:
model_b_sub = cameo.load_model('iYO844')

### Biomass rct
here I will add arginine and glutamate in to the biomass reaction and see what it does to the growth rate.*

In [23]:
#initially
model.optimize().objective_value

9.65918661040208

In [24]:
#first just add glutamine
with model:
    model.reactions.biomass.add_metabolites({model.metabolites.gln__L_c: -0.683})
    print ('gln addition:', model.optimize().objective_value)

gln addition: 9.690680005039457


In [25]:
#adding just arginine
with model:
    model.reactions.biomass.add_metabolites({model.metabolites.arg__L_c: -0.194})
    print ('arg addition:', model.optimize().objective_value)

arg addition: 9.703975049163983


In [26]:
#adding both 
with model:
    model.reactions.biomass.add_metabolites({model.metabolites.arg__L_c: -0.194, model.metabolites.gln__L_c:-0.683})
    print ('arg and gln addition:', model.optimize().objective_value)

arg and gln addition: 9.734984500301621


So it seems that the current biomass reation isn't blocked when these metabolites are added, and so the model can produce them in this case.

so I will re-add them into the biomass reaction here.

In [27]:
model.reactions.biomass.add_metabolites({model.metabolites.arg__L_c: -0.194, model.metabolites.gln__L_c:-0.683})
print ('arg and gln addition:', model.optimize().objective_value)

arg and gln addition: 9.73498450030162


I've also previously noticed that the biomass reaction has an strange ADP/ATP inclusion. The reaction produces 104.9856 mmol/gdw/h of ADP and Pi, but doesn't consume that amount of ATP. So I will add that here and see what the effect is. At a later stage we will revisit the whole biomass reaction anyway, and so will just use the coefficient here.

In [28]:
model.reactions.biomass.add_metabolites({model.metabolites.atp_c: -104.9974})
print ('fixing atp:', model.optimize().objective_value)

fixing atp: 28.828238948514628


It is very wierd that adding ATP to the biomass reaction increases growth. This indicates that somewhere we are generating huge amounts of ATP. Which is even stranger considering that there is no ATP synthase in the model.

In [29]:
#save & commit
cobra.io.write_sbml_model(model,'../model/p-thermo.xml')

## Removing RNA node
here I will remove the RNA node and see which biomass components now become problematic in hampering biomass growth. Then later I will do this also for the DNA node and then we can find a plan of how to tackle these issues hopefully . 

In [30]:
model.metabolites.RNA_c.summary()

In [31]:
remove = model.metabolites.RNA_c.reactions

In [32]:
model.remove_reactions(remove)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\group.py:110: UserWarning:

need to pass in a list



In [33]:
model.optimize().objective_value

26.4659154348342

Removing RNA doesn't really change anything, now that I've modified the biomass reaction by adding gln, arg and atp back in as they should be. If you consider the reactions that were involved in RNA: it converted ATP to UTP and UDP, indicating there is some kind of ATP surplus. Increasing ATP consumption needed for biomass results in this sink not being crucial anymore and so it can be removed.

## Removing DNA
Similar to the RNA node, I will try to remove the DNA node and see what the effect is on the model. If biomass is totally prevented i will try to figure out what metabolite is causing that blockage.

Essentially, this node is just used to convert dgtp to dctp. 

In [34]:
model.metabolites.dna_c.summary()

,,PERCENT,FLUX,REACTION_STRING
RXN_STAT,ID,,,
PRODUCING,DGTPT,100.0,34.436073,dgtp_c <=> dna_c + ppi_c
CONSUMING,DCTPT,100.0,34.436073,dctp_c <=> dna_c + ppi_c


In [35]:
remove = model.metabolites.dna_c.reactions

In [36]:
model.remove_reactions(remove)

In [37]:
model.optimize().objective_value

2.3976514947827195e-17

In [38]:
#save&commit
cobra.io.write_sbml_model(model,'../model/p-thermo.xml')

So there are some (or one) metabolite that is blocking the biomass from acculumating here. I will try to figure out which metabolite(s) are the cause of that below. 

In [37]:
#classify the biomass components
rna = ['amp_c', 'gmp_c', 'cmp_c', 'ump_c']
dna = ['damp_c', 'dgmp_c', 'dcmp_c', 'dtmp_c']
energy = ['adp_c', 'atp_c']
aa = ['ser__L_c', 'phe__L_c', 'gly_c', 'ala__L_c', 'val__L_c', 'leu__L_c', 'ile__L_c', 'thr__L_c','tyr__L_c','trp__L_c','cys__L_c','met__L_c', 'lys__L_c', 'arg__L_c', 'his__L_c', 'asp__L_c', 'glu__L_c', 'asn__L_c', 'gln__L_c', 'pro__L_c']
cofactors = ['nad_c', 'nadh_c','nadph_c', 'nadp_c', 'fad_c', 'qh2_c']
other = ['ptrc_c', 'chor_c', 'thmpp_c', 'ribflv_c', 'fe3_c', 'fe2_c', 'cl_c', 'coa_c', 'succoa_c', 'accoa_c', 'gthrd_c']

In [40]:
with model:
    for met in model.reactions.biomass.metabolites:
        if met.id in rna:
            coeff = model.reactions.biomass.metabolites[met]
            model.reactions.biomass.add_metabolites({met:-coeff})
        elif met.id in dna:
            coeff = model.reactions.biomass.metabolites[met]
            model.reactions.biomass.add_metabolites({met:-coeff})
        elif met.id in aa:
            coeff = model.reactions.biomass.metabolites[met]
            model.reactions.biomass.add_metabolites({met:-coeff}) 
#         elif met.id in energy:
#             coeff = model.reactions.biomass.metabolites[met]
#             model.reactions.biomass.add_metabolites({met:-coeff})
#         elif met.id in cofactors:
#             coeff = model.reactions.biomass.metabolites[met]
#             model.reactions.biomass.add_metabolites({met:-coeff}) 
        elif met.id in other:
            coeff = model.reactions.biomass.metabolites[met]
            model.reactions.biomass.add_metabolites({met:-coeff})  
        else: 
            continue
    solution = model.optimize()
    print (solution.objective_value)

28.67095110108862


It seems there is not a single reaction that is preventing biomass from accumulating. It is a combination of metabolites in the RNA, DNA, amino acid and other categories. I will try to figure out exactly which metabolites these are:
- RNA: ump and cmp
- DNA: dcmp and dtmp
- AA: isoleucine, threonine, lysine, arginine, aspartate, asparagine, glutamine, proline(alanine, valine, leucine, tyrosine, tryptophan, doesn't kill but decreases)
- other: putrescine

A next useful step would be to see what is the common metabolite that causes all of these products to be blocked. To test that, I will add sink reactions for each metabolite one by one and test if that 'saves' biomass accumulation. Personally, I would expect that either supplying or removing dgtp or dctp should solve the problems, as these were invovled in the DNA reaction.

In [41]:
with model:
    model.add_boundary(model.metabolites.dcmp_c, type = 'sink', reaction_id = 'test')
    model.add_boundary(model.metabolites.asp__L_c, type = 'sink', reaction_id = 'test2')
#     for met in model.reactions.biomass.metabolites:
#         if met.id in rna:
#             coeff = model.reactions.biomass.metabolites[met]
#             model.reactions.biomass.add_metabolites({met:-coeff})
#         if met.id in dna:
#             coeff = model.reactions.biomass.metabolites[met]
#             model.reactions.biomass.add_metabolites({met:-coeff})
#         if met.id in aa:
#             coeff = model.reactions.biomass.metabolites[met]
#             model.reactions.biomass.add_metabolites({met:-coeff}) 
#         if met.id in other:
#             coeff = model.reactions.biomass.metabolites[met]
#             model.reactions.biomass.add_metabolites({met:-coeff})  
#         else: 
#             continue
    solution = model.optimize()
    print ('test flux:', solution['test'])
    print ('test2 flux:', solution['test2'])
    print (solution.objective_value)

test flux: -11.940052751809935
test2 flux: -139.1551242655905
26.89552368169512


Results:
- adding dcmp_c: only blockage in amino acids
    - dna and rna metabolites become fine
    - also putrescince can be made
- adding dgtp_c: only block in DNA
    - aa, putrescine and RNA are fine
- adding glutamine: only block in DNA
    - solves block at RNA, putrescine and to some extent also amino acids. 
- adding aspartate: only block in DNA
    - sovles block at RNA, putrescine and amino acids

So it seems that there are two nodes of bloackage: 1) adding dctp/dcmp solves the problem in the DNA node, 2) adding some amino acids solves the AA problem. Adding both the dcmp and the glutamine or arginine solves all of the problems.

So I need to look into two things: cmp synthesis and amino acid synthesis (Specifically glutamine and arginine) 

I will need to check these pathways. Though, note that the more metabolites I remove from the biomass equation, the more dcmp is taken up. So there are likely other problems as well that will play in. But solving this problem first would be good, then we can go back to the other changes I've made and see if the majority of these have then been resolved too. 
    


## dCMP metabolism

In [42]:
model = cobra.io.read_sbml_model('../model/p-thermo.xml')

In [43]:
with model:
    model.add_boundary(model.metabolites.dcmp_c, type = 'sink', reaction_id = 'test')
    for met in model.reactions.biomass.metabolites:
#         if met.id in rna:
#             coeff = model.reactions.biomass.metabolites[met]
#             model.reactions.biomass.add_metabolites({met:-coeff})
#         if met.id in dna:
#             coeff = model.reactions.biomass.metabolites[met]
#             model.reactions.biomass.add_metabolites({met:-coeff})
        if met.id in aa:
            coeff = model.reactions.biomass.metabolites[met]
            model.reactions.biomass.add_metabolites({met:-coeff}) 
#         elif met.id in energy:
#             coeff = model.reactions.biomass.metabolites[met]
#             model.reactions.biomass.add_metabolites({met:-coeff})
#         elif met.id in cofactors:
#             coeff = model.reactions.biomass.metabolites[met]
#             model.reactions.biomass.add_metabolites({met:-coeff}) 
#         if met.id in other:
#             coeff = model.reactions.biomass.metabolites[met]
#             model.reactions.biomass.add_metabolites({met:-coeff})  
        else: 
          continue
    solution = model.optimize()
    print ('dcmp_c flux:', solution['test'])
    print (solution.objective_value)

dcmp_c flux: -6.890210894860734
27.56723845098847


Pyrimidines are synthesized from orotate that is coupled to a phosphorylated ribose unit. This results in the formation of UMP, which can be interconverted into CMP, dCMP and dUMP, as well as the other phosphorylated versions of these molecules. Currently to be able to accumulate biomass, our model would need the consumption of dCMP. Therefore, we need to check the pathway that will produce this metabolite. I've previously looked into the individual interconversions of the nucleotides and so for now I will assume these are correct. 

One thing I haven't looked into yet is this pathway converting asparate and carbamoly-P into orotate and further into UMP. Here I will do so and make changes that are needed.

A quick look into the pathway indicates that there are quite some reactions that I previously observed have the wrong directionality defined in them. This prevents de novo UMP synthesis, which is problematic. 

In [44]:
model.reactions.OMPDC.bounds = (-1000,0)

I looked more into the node converting dihydro-orotate into orotate. There I think that the automatic annotation is wrong. There is more indication that the strains DHORD reaction uses  NAD(H) as electron acceptors, instead of fumarate and succinate. Therefore, I will modify this reaction to reflect that. 

In [45]:
model.reactions.DHORDfum.id = 'DHORD6'

In [46]:
model.reactions.DHORD6.bounds = (0,1000)

In [47]:
model.reactions.DHORD6.notes = {}
model.reactions.DHORD6.annotation = {}

In [48]:
model.reactions.DHORD6.annotation['kegg.reaction'] = 'R01869'
model.reactions.DHORD6.annotation['sbo'] = 'SBO:0000176'
model.reactions.DHORD6.annotation['ec-code'] = 'EC 1.3.1.14'

In [49]:
model.reactions.DHORD6.add_metabolites({model.metabolites.fum_c:1, model.metabolites.succ_c:-1, model.metabolites.nad_c:-1, model.metabolites.nadh_c:1, model.metabolites.h_c:1})

In [50]:
model.reactions.DHORD6

Reaction identifier,DHORD6
Name,R01867
Memory address,0x023278ace188
Stoichiometry,dhor__S_c + nad_c --> h_c + nadh_c + orot_c (S)-Dihydroorotate + NAD+ --> H+ + NADH + Orotate
GPR,RTMO04130 or RTMO04129
Lower bound,0
Upper bound,1000


In [51]:
#improper id
model.metabolites.hco3_c.id = 'cbp_c'

In [52]:
model.metabolites.cbp_c.name = 'Carbamoyl phosphate'
model.metabolites.cbp_c.notes = {}
model.metabolites.cbp_c.annotation = model_e_coli.metabolites.cbp_c.annotation

In [53]:
model.metabolites.hco3_b_c.id = 'hco3_c'

In [54]:
model.reactions.ASPCT.bounds = (-1000,0)

In [55]:
model.reactions.OCBT.bounds = (-1000,1000)

Now the pathway from carbamoyl-P into orotate and into UMP should be correct. Then the next step is to check how the carbamoyl-P is made. It seems the reactions are correct, except the CBPSam reaction should be changed to match the E. coli CBMKr reaction.

In [56]:
model.reactions.CBPSam.id = 'CBMKr'

In [57]:
model.reactions.CBMKr.add_metabolites({model.metabolites.hco3_c:1, model.metabolites.co2_c:-1, model.metabolites.atp_c:1, model.metabolites.adp_c:-1, model.metabolites.pi_c:-1})

In [58]:
model.reactions.CBMKr.bound = (-1000,1000)

As a next step, as cbp is made from L-aspartate and L-glutamine, maybe reversing this pathway prevents their production which is the cause of the problem? 

In [59]:
model.reactions.ASPATh.id = 'ASPTA'

In [60]:
#based on thermodynamics
model.reactions.ASPO6.bounds = (-1000,0)

In [61]:
#should be reversible
model.reactions.ASPK.bounds = (-1000,1000)

In [62]:
#around this metabolite there is a cycle going on, but it doesn't really generate anything except water? but there is still some flux missing in the summar
model.metabolites.l23dhdpico_c.summary()

,,PERCENT,FLUX,REACTION_STRING
RXN_STAT,ID,,,
PRODUCING,ASPSALY,100.0,0.043714,aspsa_c + pyr_c --> h2o_c + h_c + l23dhdpico_c
CONSUMING,L23DHDPICOLY,100.0,0.043714,2.0 h2o_c + h_c + l23dhdpico_c --> aspsa_c + p...


In [63]:
#the L23DHDPICOLY reaction (R02292) doesn't exist in kegg anymore? I will remove it.
model.remove_reactions(model.reactions.L23DHDPICOLY)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\model.py:716: UserWarning:

need to pass in a list



There seems to also be a cycle at the THDPOR and THDPPOR reactions: which would just convert nadph into nadh. In B. subtilis the  EC 1.17.1.8 reaction is shown to work predominantly with NADP(H) to result in formation of the thdp product. Therefore, I will remove the THDPOR reaction. Also, after looking into it further, the  l23dhdpico_c  metabolite should be renamed to 23dhdp_c. Also the THDPPOR can then be renamed to DHDPRy. Looking further in this pathway there are some other things that can be renamed too. 

In [64]:
#rename l23dhdpico
model.metabolites.l23dhdpico_c.id = '23dhdp_c'

In [65]:
model.metabolites.get_by_id('23dhdp_c').notes = {}
model.metabolites.get_by_id('23dhdp_c').annotation = model_e_coli.metabolites.get_by_id('23dhdp_c').annotation

In [66]:
model.reactions.THDPPOR.id = 'DHDPRy'

In [67]:
model.remove_reactions(model.reactions.THDPOR)

In [68]:
model.metabolites.nal2a6o_c.id = 'acamoxm_c'

In [69]:
model.metabolites.n6all26d_c.id = '26dapime_c'

In [70]:
model.reactions.ADAPAT.id = 'APTA1i'

In [71]:
model.reactions.DAPDA.id = 'ADPDA'

I will now look into glutamine synthesis and see if that is otherwise the problem.
If i look into the purine metabolism map, one can see a pathway the converts R1P into IMP, via glutamine. As we know there is also a problem resulting in huge fluxes in the PPP, it may be worth investigating this pathway. 


In [72]:
model.reactions.ICH.id = 'IMPC'

In [73]:
model.metabolites.C04051_c.id = '5a4icam_c'

In [74]:
model.reactions.PRAISh.id = 'PRASCSi'

In [75]:
model.reactions.AIRLIG.id = 'AIRC2'

In [76]:
#reaction incorrectly present
model.remove_reactions(model.reactions.AIZCLY)

In [77]:
model.reactions.FPRAMLIG.id = 'PRAIS'

In [78]:
model.metabolites.ampm_c.id = '4ampm_c'

In [79]:
model.reactions.AMETLY.add_metabolites({model.metabolites.dmpp_c:-1, model.metabolites.co_c:1})

In [80]:
model.reactions.FGAMALIG.id = 'PRFGS'

In [81]:
model.reactions.FPGFTh.id = 'GARFT'

In [82]:
model.reactions.GARFT.bounds = (-1000,1000)

In [83]:
model.reactions.FGFTh.bounds = (-1000,1000)

In [84]:
model.reactions.PPRGL.id = 'PRAGSr'

In [85]:
model.reactions.PRAGSr.bounds = (-1000,1000)

The pathway stops getting any flux at the point where PRPP and glutamine condense into ribosylamine-5P. So I need to look into the supply and use of both PRPP and L-glutamine.

__PRPP__

In [86]:
model.reactions.R5PPPT.id = 'PRPPS'

In [87]:
#based on thermodynamics, this reaction will be made reversible. 
#is also reversible in e_coli and b_sub models
model.reactions.PRPPS.bounds = (-1000,1000)

In [88]:
model.reactions.NNDPR.bounds = (-1000,0)

In [89]:
model.reactions.ANPRT.bounds = (-1000,0)

In [90]:
model.reactions.R1PISO.id = 'PPM'

In [91]:
model.metabolites.r1p_c.name = 'Ribose 1-phosphate'

In [92]:
#based on thermodynamics, and the E coli model.
model.reactions.PYNP1.bounds = (-1000,1000)

__Glutamine__ It seems there are quite some glutamine producing and consuming reactions, so this may not be the problem. 

__thf__
Found a somewhat duplicate reaction: THFOR with both NAD(H) and NADP(H) as co-factors. From data on the Geobacillus stearothermophilus enzyme performing this conversion at 60°C, it is used to convert thf to folate. Also this uses NADP(H) as cofactor, which also makes sense from a physiological point of view. So here I will remove the THFOR1 reaction and rename the THFOR2 rct,

In [93]:
model.reactions.MOHMT.bounds = (0,1000)

In [94]:
model.remove_reactions(model.reactions.THFOR1)

In [95]:
model.reactions.THFOR2.id = 'THFOR'

In [96]:
#bassed on thermodynamics
model.reactions.DHFR2i.bounds = (-1000,0)

__10fthf__ also seems fine.

Now that I have checked these pathways, I will check if anything has changed regarding any metabolites that are still blocked that prevent the biomass accumulation.

In [170]:
with model:
    model.add_boundary(model.metabolites.glu__L_c, type = 'sink', reaction_id = 'test')
    model.add_boundary(model.metabolites.ump_c, type = 'sink', reaction_id = 'test2')
    for met in model.reactions.biomass.metabolites:
        if met.id in rna:
            coeff = model.reactions.biomass.metabolites[met]
            model.reactions.biomass.add_metabolites({met:-coeff})
        if met.id in dna:
            coeff = model.reactions.biomass.metabolites[met]
            model.reactions.biomass.add_metabolites({met:-coeff})
        if met.id in aa:
            coeff = model.reactions.biomass.metabolites[met]
            model.reactions.biomass.add_metabolites({met:-coeff}) 
        if met.id in other:
            coeff = model.reactions.biomass.metabolites[met]
            model.reactions.biomass.add_metabolites({met:-coeff})  
        if met.id in energy:
            coeff = model.reactions.biomass.metabolites[met]
            model.reactions.biomass.add_metabolites({met:-coeff})  
        if met.id in cofactors:
            coeff = model.reactions.biomass.metabolites[met]
            model.reactions.biomass.add_metabolites({met:-coeff}) 
        else: 
            continue
    solution = model.optimize()
    print ('test flux:', solution['test'])
    print ('test2 flux:', solution['test2'])
    print (solution.objective_value)

test flux: 0.0
test2 flux: -1.850371707708594e-17
0.0


Doing the above changes has changed quite some stuff in the model. Now supplying dcmp and gln no longer rescues the growth: you can also see no flux being taken up to restore growth in any case. Also the model has now become additionally sensitive to some component in the energy, other and cofactors group. 

- Energy block: Supplying atp to the model solves the problem in the energy category, meaning that somewhere we do not produce enough ATP, though the uptake flux is only very small, so our model can generate quite some atp.
- co-factor group: solved with adding fad
- amino acid group: solved with adding glutamate, the critical metabolite coupling many amino acids to the TCA, so atleast the interconversions see to be oke. 
- others: now the model has blocked ptrc, ribflv, coa, accoa and succoa metabolites. 
    - supplementing with coa solves the accoa and succoa problem
    - supplementing with glu__L solves the ptrsc problem
    - need to supplement with ribflv
- DNA/RNA: still multiple problems now... I assume this comes from the fact that we cannot produce all amino acids, as these are inputs for the nucleotide pathways. but supplementing glu which solves the aa problem doesnt solve this. So I will need to look into this more later. 

In [164]:
#save and commit, even though it doesn't grow I will keep track of the steps in this branch
cobra.io.write_sbml_model(model,'../model/p-thermo.xml')

### FAD metabolism
Looking at all the FAD reactions in the model, the problem seems to be that most are irreversible to consume FAD. There are not really any options that produce fad, except two which have been left reversible. So I need to check that these reversabilities are correct and if there are other fad producing reactions.

In [300]:
model = cobra.io.read_sbml_model('../model/p-thermo.xml')

In [5]:
#based on thermodynamics this should be the reverse direction
# to be careful not to over-constrict i will just make them irreversible, as this is also the approach used in e. coli for these reactions
model.reactions.PPCOAOm.bounds = (-1000,1000)

In [6]:
#change id and bounds
model.reactions.ACOA160OR.id = 'ACOAD7f'

In [7]:
model.reactions.ACOAD7f.bounds = (-1000,1000)

In [8]:
#rename and change bounds
model.reactions.ACOA140OR.id = 'ACOAD6f'

In [9]:
model.reactions.ACOAD6f.bounds = (-1000,1000)

In [10]:
#along the same rationale as above
model.reactions.DECOT.bounds = (-1000,1000)

In [11]:
model.reactions.ACOA80OR.id = 'ACOAD3f'

In [12]:
model.reactions.ACOAD3f.bounds = (-1000,1000)

In [13]:
model.reactions.HEXOT.bounds = (-1000,1000)

In [14]:
model.reactions.DMALRED.id = 'MDH4'

In [15]:
model.reactions.MDH4.bounds = (0,1000)

In [16]:
model.reactions.ACOA40OR.id = 'ACOAD1f'  

In [17]:
model.reactions.ACOAD1f.bounds = (-1000,1000)

In [18]:
model.reactions.ACOA120OR.id = 'ACOAD5f'

In [19]:
model.reactions.ACOAD5f.bounds = (-1000,1000)

In B. subtilis, the model contains a different node for the succinate dehydrogenase. The conversion of succinate to fumarate donates electrons to FAD, after which the formed fad passses the electrons on towards a quinone pumping 2 protons into of the cell. This is also similar to what the E. coli model contains: e. coli has transhydrogenases that can interconvert the co-factors. Our strain likely does not contain this, and so the B. subtilis pathway seems to make more sense for our strain too. 

In [20]:
model.add_reaction(Reaction(id='SUCD5'))

In [21]:
model.reactions.SUCD5.name = 'Succinate dehydrogenase'

In [22]:
model.reactions.SUCD5.add_metabolites({model.metabolites.fadh2_c:-1, model.metabolites.ubiquin_c: -1, model.metabolites.fad_c:1, model.metabolites.qh2_c: 1, model.metabolites.h_e: -2, model.metabolites.h_c:2})

Now that I've gone through all of the FAD containing reactions, let me check if it is still a blocked reaction in our model. 

In [220]:
with model: #test supplementing different metabolites if it removes the block
    #model.add_boundary(model.metabolites.dtmp_c, type = 'sink', reaction_id = 'test')
#     model.add_boundary(model.metabolites.atp_c, type = 'sink', reaction_id = 'test2')
    for met in model.reactions.biomass.metabolites:
        if met.id in rna:
            coeff = model.reactions.biomass.metabolites[met]
            model.reactions.biomass.add_metabolites({met:-coeff})
        if met.id in dna:
            coeff = model.reactions.biomass.metabolites[met]
            model.reactions.biomass.add_metabolites({met:-coeff})
        if met.id in aa:
            coeff = model.reactions.biomass.metabolites[met]
            model.reactions.biomass.add_metabolites({met:-coeff}) 
        if met.id in other:
            coeff = model.reactions.biomass.metabolites[met]
            model.reactions.biomass.add_metabolites({met:-coeff})  
        if met.id in energy:
            coeff = model.reactions.biomass.metabolites[met]
            model.reactions.biomass.add_metabolites({met:-coeff})  
        if met.id in cofactors:
            coeff = model.reactions.biomass.metabolites[met]
            model.reactions.biomass.add_metabolites({met:-coeff}) 
        else: 
            continue
    solution = model.optimize()
    #print ('test flux:', solution['test'])
#     print ('test2 flux:', solution['test2'])
    print (solution.objective_value)

9.879107229943916


Shit, for some reason it still doesn't de-block the reaction. 

In the version of the model before the nodes were removed, the fad is generated from fmn.
When I add fmn and atp, then we do get formation of fad. The only problem is energy here. So it seems we do not produce enough ATP for the formation of FAD. This is something we will re-visit some other time anyway. So for now I will consider the fad production fixed.

The other nodes are still blocked even with the addition of energy, so i will move on to the next node now.


In [41]:
#save & commit
cobra.io.write_sbml_model(model,'../model/p-thermo.xml')

### Ribflv node
Ribflav is made from ru5p mainly. But there is one other metabolite that is consumed in the pathway. When we supply 4r5au externally, we can grow again without needing ribflav. So I will figure out how this is synthesized.

In [72]:
model.reactions.AUS.id = 'PMDPHT'

In [76]:
model.reactions.APRAUR.bounds = (0,1000)

The 4rau5 metabolite is formed in a cascade that descends from the degradation of gtp. As the RNA node is also still a problem, once we fix that, the ribflv should also be fixed. So I will leave this for now and move on to the next step.

In [84]:
#save&commit
cobra.io.write_sbml_model(model,'../model/p-thermo.xml')

### CoA node

Adding atp also solves the problem here. But I will go through the CoA production cascade to make sure it is properly produced anyway. 

In [209]:
model = cobra.io.read_sbml_model('../model/p-thermo.xml')

In [210]:
model.remove_reactions(model.reactions.APPAT)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\model.py:716: UserWarning:

need to pass in a list

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\group.py:110: UserWarning:

need to pass in a list



In [211]:
model.reactions.DPR.bounds = (-1000,0)

In [212]:
model.reactions.BCTA_val_h.id = 'VALTA'

In [213]:
#duplicate metabolite: remove dhmb_c and its reactions
model.remove_reactions(model.reactions.DHMBHL)

In [214]:
model.remove_reactions(model.reactions.DHMBISO)

In [215]:
model.remove_metabolites(model.metabolites.dhmb_c)

After these changes, it shows that we can produce coa. The pathway just requires quite some ATP to make coa, and so it seems this is the bottleneck preventing coa synthesis. If we supply coa, we also prevent the blockage to accoa. But we do still have the succoa node being blocked.. Also supplying atp doesn't solve the succoa problem. So I will just check this here. 

__Succoa__

If I supply akg, then the succoa is no longer blocked. So look into akg production

In [216]:
#based on thermodynamics
model.reactions.ALDD2x.bounds = (0,1000)

In [217]:
model.reactions.MMM2.bounds = (0,1000)

In [219]:
model.reactions.ABTA.bounds = (0,1000)

In [220]:
model.reactions.BCTA_glu_h.id = 'LEUTAi'

In [221]:
model.reactions.ACOTA.bounds = (-1000,1000)

In [222]:
model.reactions.ICDHyr.bounds = (-1000,1000)

In [223]:
model.reactions.BCTA.id = 'ILETA'

In [230]:
model.reactions.AOOA.id = 'HSTPT'

In [231]:
model.reactions.PSAT.id = 'PSERT'

In [232]:
model.reactions.PSERT.bounds = (0,1000)

In [ ]:
#duplicate metabolites cyst_c  and cyst__L_c and hence some duplicate reactions cycling.

In [254]:
model.reactions.CBL.id = 'CYSTL'

In [255]:
model.remove_reactions(model.reactions.CYSTHI)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\model.py:716: UserWarning:

need to pass in a list

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\group.py:110: UserWarning:

need to pass in a list



In [258]:
model.remove_reactions(model.reactions.CYSTT)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\model.py:716: UserWarning:

need to pass in a list

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\group.py:110: UserWarning:

need to pass in a list



In [259]:
model.remove_metabolites(model.metabolites.cyst_c)

In our model we have an oxalosuccinate metabolite currently. This is normally made from isocitrate and then converted further into akg. Therefore this reaction is overall the same as the ICDHyr reaction, and to make sure the flux is properly shown I will remove the two osuc reactions and also the metabolite itself.

In [319]:
model.remove_reactions(model.reactions.ISOCITOR)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\model.py:716: UserWarning:

need to pass in a list

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\group.py:110: UserWarning:

need to pass in a list



In [320]:
model.remove_reactions(model.reactions.OAACOLY)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\model.py:716: UserWarning:

need to pass in a list

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\group.py:110: UserWarning:

need to pass in a list



In [321]:
model.remove_metabolites(model.metabolites.osuc_c)

After these changes, it seems that there is a functional TCA cycle, which is good. But somehow our succoa metabolite is still blocked. 

It seems there is a carbon source input somehow missing: adding any TCA intermediate, or amino acid solves this problem. Energy itself doesn't seem to be the problem here. I will just make a note of this and move on, as I'm quite confused exactly where this issue comes from but it has to do with carbon input into the TCA cycle i think. 

In [346]:
#save&commit anyway
cobra.io.write_sbml_model(model,'../model/p-thermo.xml')

### Putrescine
Putrescine comes downstream from proline, so here i just check those pathways to be sure that as long as we have proline we should get putrescine.

In [11]:
model = cobra.io.read_sbml_model('../model/p-thermo.xml')

In [12]:
model.reactions.APAPTm .id = 'SPMS'

In [13]:
#PTRCTA is a wrong annotation, it should be R08714 

In [14]:
model.reactions.PTRCTA.id = 'PTRCTA2'

In [15]:
model.reactions.PTRCTA2.name = 'putrescine:pyruvate aminotransferase'
model.reactions.PTRCTA2.notes = {}
model.reactions.PTRCTA2.annotation = {}
model.reactions.PTRCTA2.annotation['sbo'] = 'SBO:0000176'
model.reactions.PTRCTA2.annotation['kegg.reaction'] = 'R08714'
model.reactions.PTRCTA2.annotation['ec-code'] = '2.6.1.113'

In [16]:
model.reactions.PTRCTA2.add_metabolites({model.metabolites.akg_c:1, model.metabolites.glu__L_c:-1, model.metabolites.pyr_c:-1, model.metabolites.ala__L_c:1})

In [17]:
model.reactions.PTRCTA2.bounds = (-1000,1000)

In [18]:
model.reactions.G5DHx.id = 'G5SD'

In [19]:
model.reactions.G5SADrm.id = 'G5SADs'

In [20]:
model.reactions.G5SADs.bounds = (0,1000)

Conversion from pyr5c_c to glu__L_c: According to data shown in G. thermodenitirifcans and G. stearothermophilus the cofactor used here is NAD(H). Alos, then the reaction should not be irreversible.

In [21]:
model.remove_reactions(model.reactions.PYR5COR)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\model.py:716: UserWarning:

need to pass in a list

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\group.py:110: UserWarning:

need to pass in a list



In [22]:
model.reactions.P5CD.bounds = (0,1000)

In [23]:
model.remove_reactions(model.reactions.P5CRx)

In [24]:
model.reactions.P5CR.bounds = (-1000,0)

In [25]:
model.reactions.LPROQOR.bounds = (0,1000)

With these changes, putrescine should be connected to proline. This is also a blocked metabolite somewhere upstream and so will be revisited. If we supply proline to the model, now putrescine can be made and that pathway is no longer blocked. 

In [26]:
#save&commit
cobra.io.write_sbml_model(model,'../model/p-thermo.xml')

## Amino acid node
Maybe before I can really see anything here I should go through each amino acid and check that there are not two seperate versions, and if there are there actually should be. 

What I will also focus on in this node is the nitrogen incorporation into the molecules. It is known that glutamine and glutamate are the two amino acids that are formed by incorporating ammonia, and they donate the amine group (indirectly) to all other amino acids. Therefore the reactions of other amino acids that assimilate nitrogen into their molecules should be removed, or at least the bounds changed to not assimilate nh3. (see Walker at al. 2016, DOI doi:10.1007/s10295-015-1665-y ) 

In [199]:
model = cobra.io.read_sbml_model('../model/p-thermo.xml')

In [200]:
#other observations
model.reactions.RBFSb.bounds = (0,1000)
model.reactions.DLDLBT.bounds = (0,1000)

__Threonine__

In [201]:
model.metabolites.thrp_c.name = 'L-Threonine phosphate'

threonine phosphate is used in the formation of vitamin B12. As *P. thermoglucosidasius NCIB11955* is known to be able to produce vitamin B12, together with the fact that there is a putative threonine-protein kinase annotated in the genome means that one could expect this conversion to be present. Therefore I will add this reaction to fill the gap in this pathway,

In [202]:
model.add_reaction(Reaction(id='LTHRK'))

In [203]:
model.reactions.LTHRK.name = 'L threonine kinase'
model.reactions.LTHRK.annotation['kegg.reactions'] = 'R07341'
model.reactions.LTHRK.bounds = (0,1000)
model.reactions.LTHRK.annotation['sbo'] = 'SBO:0000176'
model.reactions.LTHRK.annotation['ec-code'] = '2.7.1.77'

In [204]:
model.reactions.LTHRK.add_metabolites({model.metabolites.thr__L_c:-1, model.metabolites.atp_c:-1, model.metabolites.thrp_c:1, model.metabolites.adp_c:1})

In [205]:
#remove reaction
model.remove_reactions(model.reactions.HSDx)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\model.py:716: UserWarning:

need to pass in a list

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\group.py:110: UserWarning:

need to pass in a list



In [206]:
model.reactions.HSDH.id = 'HSDy'

In [207]:
model.reactions.ASPSALY.id = 'DHDPS'

In [208]:
model.reactions.DHDPS.bounds = (0,1000)

In [209]:
model.reactions.ASAD.bounds = (-1000,1000)

In [210]:
#connected to aspartate now

__Alanine__
There are three alanine versions in the model: L, D (both alpha) and beta. I will check that they should all be in the model and also ensure that the correct reactions are associated to them.

In [211]:
#only problem of ala__D
model.reactions.ALATA_D.bounds = (-1000,1000)

In [212]:
#wrong direction
model.reactions.ALAD_L.bounds = (-1000,0) 

In [213]:
model.reactions.LCTST.bounds = (-1000,1000)

The B. subtilis genome has been shown to encode an L-alanine transaminase, reflected by the b. sub model reaction ALATA_L. Performing a Blast on this protein gave multiple significant hits. As it seems that our current model doesn't contain the alanine synthesis route known to be functional in B. sub (and E. coli) it should be added. To do so, I will add the ALATA_L reaction.

In [214]:
model.add_reaction(Reaction(id='ALATA_L'))

In [215]:
model.reactions.ALATA_L.name = 'L-alanine transaminase'

In [216]:
model.reactions.ALATA_L.annotation= model_e_coli.reactions.ALATA_L.annotation

In [217]:
model.reactions.ALATA_L.add_metabolites({model.metabolites.akg_c:-1, model.metabolites.ala__L_c:-1, model.metabolites.glu__L_c:1, model.metabolites.pyr_c:1})

In [218]:
model.reactions.ALATA_L.bounds = (-1000,1000)

In [219]:
#next look into b-alanine

In [220]:
#knowing the physiological role of NADPH as electron donor  in anabolism, it makes m ore sense  that this reaction is in the reverse
model.reactions.MALCOAOR.bounds = (-1000,0)

I will leave both MMSAD3 and MSAOR for now. They are the same reaction but with other co-factors. As they atleast do not allow a cylce to form exchanging the electrons, then I think it is oke to keep. 

In [221]:
model.reactions.DURAD.bounds = (-1000,0)

In [222]:
#connected to glutamate

__Phenylalanine__ In our model we have both D and L version. Both can be made and degraded in the same way from other D or L amino acids, so I will leave them for now. 

In [223]:
#phe__L_c should only get nitrogen from glutamate, so will prevent that if PHEOR
model.reactions.PHEOR.bounds = (0,1000)
model.reactions.PHEAOR.bounds = (0,1000)

In [225]:
#connected to D- or L-glutamate

In [226]:
#save& commite
cobra.io.write_sbml_model(model,'../model/p-thermo.xml')

__Glycine__

In [345]:
model = cobra.io.read_sbml_model('../model/p-thermo.xml')

In [346]:
#missing a metabolite
model.reactions.GHMT.add_metabolites({model.metabolites.thf_c:-1, model.metabolites.h2o_c:2})

In [347]:
#remove GLYO1, causes a cycle making NAHD and oxygen
model.remove_reactions(model.reactions.GLYO1)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\model.py:716: UserWarning:

need to pass in a list

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\group.py:110: UserWarning:

need to pass in a list



In [348]:
model.reactions.PRAGSr.bounds = (0,1000)

In [349]:
#coupled to serine

__Serine__

In [350]:
model.reactions.SERH.id = 'TRPS1'

In [351]:
model.reactions.TRPS1.bounds = (0,1000)

In [352]:
model.reactions.PSP.id = 'PSP_L'

In [353]:
model.reactions.PGCD.bounds = (0,1000)

In [354]:
#coupled to glycerate and glutamate

__Valine__

In [355]:
#only nitorgen comes from glutamate
model.reactions.VALDHr.bounds = (-1000,0)

A problem here comes from the formation of acetolactate from pyruvate. The model should use ACLS but instead uses the APL reaction that seems to be wrongly incorporated into the model. Therefore I will remove it here.

In [356]:
model.remove_reactions(model.reactions.APL)

In [357]:
#coupled to pyruvate and glutamate

In [358]:
#save&commit
cobra.io.write_sbml_model(model,'../model/p-thermo.xml')

__Leucine__

In [467]:
model = cobra.io.read_sbml_model('../model/p-thermo.xml')

In [468]:
model.reactions.LLEUDr.bounds = (0,1000)

In [469]:
#CO2 on the wrong side of the reaction
model.reactions.OMCDC.add_metabolites({model.metabolites.co2_c:2, model.metabolites.h_c:-1})

In [470]:
model.reactions.IPMD.bounds = (0,1000)

In [471]:
model.reactions.IMDHT_3c2hmp.id = 'IPPMIa'

In [472]:
model.reactions.IMDHT.id = 'IPPMIb'

In [473]:
#connected to pyruvate

__Isoleucine__

In [474]:
model.reactions.ILEOR.bounds = (0,1000)

In [475]:
#linked to pyruvate & glutamate

__Tyrosine__

In [476]:
model.reactions.CHORM.bounds = (0,1000)

In [477]:
model.reactions.ADCS.bounds = (0,1000)

In [478]:
#i think reaction should not function in this direction, as the nitrogen in themolecules should come from glu/gln. 
#so Ill change the bounds so this is a degradation reaction.
model.reactions.ANS.bounds = (-1000,0)

In [479]:
#went through shiki pathway
#coupled to glu/gln and e4p and pep.

__Tryptophan__

In [480]:
model.reactions.G3PL.id = 'TRPS3'

In [481]:
model.reactions.TRPS3.bounds = (0,1000)

In [482]:
model.reactions.ANS.bounds = (0,1000)

In [483]:
#again ammonia assimilationg shouldnt be possible like this
model.reactions.ANS2.bounds = (-1000,0)

In [484]:
#linked to gln and chorismate, whose pathway has been checkd above.

In [485]:
#save&commit
cobra.io.write_sbml_model(model,'../model/p-thermo.xml')

__Cysteine__

In [591]:
model = cobra.io.read_sbml_model('../model/p-thermo.xml')

In [592]:
model.reactions.SAT.id = 'SERAT'

In [593]:
#coupled to serine

__Methionine__


In [594]:
model.reactions.MATm.id = 'METAT'

In [595]:
model.reactions.METAT.bounds = (-1000,0)

In [596]:
model.reactions.MTHFO.bounds =  (-1000,0)
model.reactions.MTHFO_nadp.bounds = (-1000,0)

In [597]:
#linked to cysteine

__Lysine__

In [598]:
model.reactions.UAMAGLIG.id = 'UAAGDS'

In [599]:
model.metabolites.uaagmd_c.id = 'ugmd_c'

In [600]:
model.reactions.ALAALALIG.id = 'UGMDDS'

In [601]:
#linked to aspartate

__Arginine__

In [602]:
model.metabolites.acg5sa_c.name = 'N-Acetyl-L-glutamate 5-semialdehyde'

In [603]:
model.reactions.AGGPR.id = 'AGPR'

In [604]:
model.reactions.ACGS.bounds = (0,1000)

In [605]:
#linked to glutamate

In [606]:
#save&commit
cobra.io.write_sbml_model(model,'../model/p-thermo.xml')

__Histdine__ 
I found a duplicate of histidine: lhistidi_c. I will remove it and its reactions. Maybe more to be removed here? check the normal pathway first

In [668]:
model = cobra.io.read_sbml_model('../model/p-thermo.xml')

In [669]:
model.reactions.HDH.id = 'HISTD'

In [670]:
model.remove_reactions(model.reactions.HISOR)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\model.py:716: UserWarning:

need to pass in a list

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\group.py:110: UserWarning:

need to pass in a list



In [671]:
model.remove_reactions(model.reactions.HISTIOR)

In [672]:
model.remove_metabolites(model.metabolites.lhistidi_c)

In [673]:
model.reactions.PRICIh.id = 'PRMICI'

In [674]:
model.reactions.PRMICI.bounds = (-1000,1000)

In [675]:
model.reactions.PRACHh.id = 'PRAMPC'

In [676]:
model.reactions.PRADPh.id = 'PRATPP'

In [648]:
#linked to r5p and gln/glu.

__Aspartate__

Everything with aspartate seems fine.

__Asparagine__ Also seems correct.

__Proline__
already linked to arginine

In [677]:
#save&commit
cobra.io.write_sbml_model(model,'../model/p-thermo.xml')

###  Glutamate and glutamine

Here I will first look into the D-versions of these two amino acids to get them sorted. Then afterwards I will look into the L-verions and look into literature about what is known of glutamine and glutamate metabolism in (thermophilic) bacilli. 

NADPH dependent GDH is used to assimilate nitrogen into a-ketoglutarate, forming glutamate. The NADH dependent GDH is used to catabolize glutamate into a-ketoglutarate when glutamate is carbon and nitrogen source.

However currently, both the NADPH and NADH dependent GDH enzymes in our model are reversible. As a result, the model just cycles the fluxes between these two, resulting in the transfer of electrons from NADH to NADPH, which is energetically not so reasonable. Also it will cause us to bias the cofactor balances we have in the cell. Even if we fix the two reactions in the direction they are expected to work, this keeps happening. In most bacteria it has been shown that either the one or the other is present and that they are relatively stringent with co-factor specificity.

That being said, I looked more into the GDH enzymes that have been found in bacilli. It seems most bacilli (and shown for one geobacillus) that there is no NADP(H) dependent GDH. Hence the GDH functions to catabolize glutamate into the TCA when needed (as you also mentioned is seen).

Instead of assimilating nitrogen via GDH, it seems bacilli use the GOGAT reaction: which transfers a nitrogen from glutamine to a-ketoglutarate to form two glutamate molecules. A glutamate synthase (GS) can convert glutamate into glutamine by incorporating NH4 and using some ATP. (see picture below for a visual)

All of that being said, I think the GS/GOGAT assimilation route is more likely to be expected for our strain, with the GDH is being used for glutamate catabolism.


Literature: 
- Commichau 2008, doi:10.1128/JB.00099-08
- Khan, Biosci. Biotechnol. Biochem., 69 (10), 1861-1870, 2005
- Schmidt, Journal of General Microbiology (1982), 128, 17 13-1 7 18.


On another note, the further research here has shown that in bacilli it has been postulated that alanine can also be formed by nitrogen assimilation, and so the ALAD reactions should be reversible.

In [5]:
model = cobra.io.read_sbml_model('../model/p-thermo.xml')

In [6]:
model.reactions.ALAD_L.bounds = (-1000,1000)

__D-glutamate__ Seems fine

__D-glutamine__ This has been incorrectly added to the model. So I will remove this metabolite and the one reaction associated to it.

In [7]:
model.remove_reactions(model.reactions.GLNAMHYDRO)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\model.py:716: UserWarning:

need to pass in a list

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\group.py:110: UserWarning:

need to pass in a list



In [8]:
model.remove_metabolites(model.metabolites.gln__D_c)

In [9]:
#GLUS_nadph  is the GOGAT, or GLUSy in the e. coli model
model.reactions.GLUS_nadph.id = 'GLUSy'

In [10]:
#GALh is the glutamine synthase
model.reactions.GALh.id = 'GLNS'

In [11]:
#problematic reaction: GAM. it can just assimilate nitrogen without needing any energy. 
model.remove_reactions(model.reactions.GAM)

In [12]:
model.metabolites.ru5p__D_c.name= 'D-Ribulose 5-phosphate'

In [13]:
model.metabolites.pydx5p_c.name = 'Pyridoxal phosphate'

In [14]:
#based on thermodynamics
model.reactions.PYDXS.boudsn = (0,1000)

In [15]:
model.metabolites.glu5sa_c.name = 'L-Glutamate 5-semialdehyde'

In [16]:
#based on thermodynamics
model.reactions.PUTA3.bounds = (0,1000)

Now I will try to build in the glutamate/glutamine pathway as I would expect it to be present, as described above.

In [17]:
model.remove_reactions(model.reactions.GDH_nadp)

In [18]:
model.reactions.GDH.bounds = (0,1000)

In [19]:
model.remove_metabolites(model.metabolites.glc__bD_c)

In [20]:
#save&commit
cobra.io.write_sbml_model(model,'../model/p-thermo.xml')

with these changes, and supplying a-ketoglutarate, all amino acids are no longer blocked, which is good. But suppling accoa does not restore the biomass. This again seems to indicate there is something wrong further upstream of the pathways.

### DNA/RNA node
it seems that the synthesis of all nucleotides and deoxy-nucleotides are blocked in the model right now. I need to figure out what upstream metabolite(s) cause these blocks.

For the purines, IMP is the common metabolite that is used to form the other purines (A and G). IMP comes from PRPP and Glutamine. I've checked the pathay from these precursors to IMP before. 

For the pyrimidines (U,T,C) can all be synthesizied from UMP. The UMP is made from glutamine and aspartate. This part of the pathway I have checked also. 

For some reason I can't find a supplementation that fixes all of the nucleotides: even adding both imp and ump doesnt resolve the problem.
So I will look into the purines and pyrimidines seperately to find out what is missing in this synthesis.

__dAMP from AMP__
dADP is made from ADP, using the trdr cofactor. The reason we cannot form dADP here is because the trdrd_c metabolite cannot be regenerated in any reaction we have currently. This is a problem and prevents flux throught the pathway. This is also the cofactor used to generate dgdp, dudp and dcdp too. So if we solve this regeneration then we may have fixed a part of the blockage problem in this pathway. 

This should allow damp production from atp or imp. This seems to also have solved the dependency for dgmp, as it goes via the same mechanism and co-factor, trdrd_c. 

In [21]:
model.reactions.TDSRh.id = 'TRDR'

In [22]:
model.reactions.TRDR.bounds = (-1000,0)

With fixing this cofactor, we have now fixed the blockage in the DNA and RNA pathways. When IMP and UMP are supplied, we can generate all deoxy- and nucleotides. So now I have to figure out where the upstream blockage is. 

__IMP pathway__ IMP comes from a pathway that starts with pram_c, which is made from glutamine and prpp. Replacing IMP with pram_c kills the production of purines, so there is still something wrong in this pathway.

Supplying metabolite 5caiz_c still allows formation of all purines. however one step up the pathway, supplying air_c doesn't rescue growth. So something seems to be the problem in converting air_c into 5caiz_c., i.e. reaction AIRC2. If we supply both hco3 and air, then the purines can be made again. So it seems something is wrong with the bicarbonate supply.

Upon inspection, we see that there are only hco3 consuming reactions, no producing reactions possible, explaining the imbalance here. We are missing a reaction that allows co2 to solubilize to hco3 reversibly. This reaction will be added.

In [23]:
model.add_reaction(Reaction(id='HCO3E'))

In [24]:
model.reactions.HCO3E.name = 'HCO3 equilibration reaction'

In [25]:
model.reactions.HCO3E.annotation = model_e_coli.reactions.HCO3E.annotation 

In [26]:
model.reactions.HCO3E.add_metabolites({model.metabolites.co2_c:-1, model.metabolites.h2o_c:-1, model.metabolites.h_c:1, model.metabolites.hco3_c:1})

In [27]:
model.reactions.HCO3E.bounds =(-1000,1000)

With these changes, we can produce all purines from ribose-5-phosphate, prpp or glutamine. So it seems this blockage has now been resovled!

__UMP pathway__
For the production of the pyrimidines we now can synthesize them all from UMP, but now need to see where upstream there is a block that prevents the production of the pyrimidines from carbamoyl-phosphate and then arginine and glutamine. 

After doing the fixes above, we see that the pyrimidines are also no longer blocked and require no supplementation. So it seems that there was a common problem there that has been fixed. 
This block has then been completely resolved!

In [28]:
#save and commit
cobra.io.write_sbml_model(model,'../model/p-thermo.xml')

# conclusion
Initially we saw various causes for the blockes in biomass generation, however after having gone through each category we need to revisit what nodes are still blocked and what could be the reason. After fixing the purine and pyrimidine block for example, we also alleviated the FAD and Ribflv block (cause by blocked atp and gtp respectively). Also the energy is no longer such a problem as we can produce both adp and atp in purine biosynthesis now. 

Supprisingly, fixing the nucleotides also resolved the 'other' category: the succoa, putrescine and ribflav are no longer an issue. Also now the amino acids are not blocked.

Going back to the normal model, just growing on glucose, we get a growth rate of 1.84 which is quite realistic.

Of course there are still more issues with the model that need revisiting, but this is already a great start to a more realistic prediction, even with the bounds opened again. 




In [144]:
rna = ['amp_c', 'gmp_c', 'cmp_c', 'ump_c']
dna = ['damp_c', 'dgmp_c', 'dcmp_c', 'dtmp_c']
purines = ['amp_c', 'gmp_c', 'damp_c', 'dgmp_c']
pyrimidines = ['cmp_c', 'ump_c', 'dcmp_c', 'dtmp_c']
energy = ['adp_c', 'atp_c']
aa = ['ser__L_c', 'phe__L_c', 'gly_c', 'ala__L_c', 'val__L_c', 'leu__L_c', 'ile__L_c', 'thr__L_c','tyr__L_c','trp__L_c','cys__L_c','met__L_c', 'lys__L_c', 'arg__L_c', 'his__L_c', 'asp__L_c', 'glu__L_c', 'asn__L_c', 'gln__L_c', 'pro__L_c']
cofactors = ['nad_c', 'nadh_c','nadph_c', 'nadp_c', 'fad_c', 'qh2_c']
other = ['ptrc_c', 'chor_c', 'thmpp_c', 'ribflv_c', 'fe3_c', 'fe2_c', 'cl_c', 'coa_c', 'succoa_c', 'accoa_c', 'gthrd_c']

In [4]:
other = ['ptrc_c', 'chor_c', 'thmpp_c', 'ribflv_c', 'fe3_c', 'fe2_c', 'cl_c', 'coa_c', 'succoa_c', 'accoa_c', 'gthrd_c']
with model:
#     model.add_boundary(model.metabolites.glc__D_c, type = 'sink', reaction_id = 'test')
#     model.add_boundary(model.metabolites.r5p_c , type = 'sink', reaction_id = 'test2')
#     model.add_boundary(model.metabolites.hco3_c, type = 'sink', reaction_id = 'test3')
#     for met in model.reactions.biomass.metabolites:
#         if met.id in purines:
#             coeff = model.reactions.biomass.metabolites[met]
#             model.reactions.biomass.add_metabolites({met:-coeff})
#         if met.id in pyrimidines:
#             coeff = model.reactions.biomass.metabolites[met]
#             model.reactions.biomass.add_metabolites({met:-coeff})
#         if met.id in aa:
#             coeff = model.reactions.biomass.metabolites[met]
#             model.reactions.biomass.add_metabolites({met:-coeff}) 
#         if met.id in other:
#             coeff = model.reactions.biomass.metabolites[met]
#             model.reactions.biomass.add_metabolites({met:-coeff})  
#         if met.id in energy:
#             coeff = model.reactions.biomass.metabolites[met]
#             model.reactions.biomass.add_metabolites({met:-coeff})  
#         if met.id in cofactors:
#             coeff = model.reactions.biomass.metabolites[met]
#             model.reactions.biomass.add_metabolites({met:-coeff}) 
#         else: 
#             continue
    solution = model.optimize()
    #print (model.metabolites.glu__D_c.summary())
    #print ('test flux:', solution['test'])
    #print ('test2 flux:', solution['test2'])
    print (solution.objective_value)

1.840490745188007


In [5]:
Builder(model = model, map_json = '../map/p-thermo-map.json', reaction_data = solution.fluxes.to_dict())

Builder(reaction_data={'IDPh': 4.154552564511286, 'CAT': 0.0, 'PDHam1hi': 0.0, 'CCP': 0.0, 'HYDA': 0.0, 'MALHY…